# Fino podešavanje Open AI modela

Ova bilježnica temelji se na trenutnim uputama iz [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) dokumentacije Open AI-a.

Fino podešavanje poboljšava performanse temeljnih modela za vašu aplikaciju tako što ga ponovno trenira s dodatnim podacima i kontekstom relevantnim za određeni slučaj upotrebe ili scenarij. Imajte na umu da tehnike oblikovanja upita poput _učenja iz nekoliko primjera_ i _generiranja uz prošireno dohvaćanje_ omogućuju poboljšanje zadanog upita relevantnim podacima radi bolje kvalitete. Međutim, ove metode su ograničene maksimalnom veličinom prozora tokena ciljanog temeljnog modela.

S finim podešavanjem zapravo ponovno treniramo sam model s potrebnim podacima (što nam omogućuje korištenje mnogo više primjera nego što stane u maksimalni prozor tokena) - i postavljamo _prilagođenu_ verziju modela koja više ne treba primjere tijekom izvođenja. Ovo ne samo da poboljšava učinkovitost našeg dizajna upita (imamo više fleksibilnosti u korištenju prozora tokena za druge stvari), već potencijalno smanjuje i troškove (smanjenjem broja tokena koje trebamo poslati modelu tijekom izvođenja).

Fino podešavanje ima 4 koraka:
1. Pripremite podatke za treniranje i prenesite ih.
1. Pokrenite posao treniranja kako biste dobili fino podešen model.
1. Procijenite fino podešeni model i ponovite postupak radi kvalitete.
1. Postavite fino podešeni model za izvođenje kada ste zadovoljni.

Imajte na umu da ne podržavaju svi temeljni modeli fino podešavanje - [provjerite OpenAI dokumentaciju](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) za najnovije informacije. Također možete fino podesiti prethodno fino podešen model. U ovom vodiču koristit ćemo `gpt-35-turbo` kao ciljani temeljni model za fino podešavanje.

---


### Korak 1.1: Pripremite svoj skup podataka

Izgradimo chatbot koji vam pomaže razumjeti periodni sustav elemenata tako što odgovara na pitanja o elementu u obliku limerika. U _ovom_ jednostavnom vodiču, samo ćemo napraviti skup podataka za treniranje modela s nekoliko primjera odgovora koji pokazuju očekivani format podataka. U stvarnom slučaju korištenja, trebali biste napraviti skup podataka s mnogo više primjera. Također biste mogli koristiti otvoreni skup podataka (za vaše područje primjene) ako postoji, i preoblikovati ga za korištenje u fino podešavanju.

Budući da se fokusiramo na `gpt-35-turbo` i želimo odgovor u jednom koraku (chat completion), možemo napraviti primjere koristeći [ovaj predloženi format](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) koji odgovara zahtjevima OpenAI chat completion modela. Ako očekujete višekratni razgovor, koristili biste [format za višekratne primjere](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) koji uključuje parametar `weight` kako bi se označilo koje poruke treba koristiti (ili ne) u procesu fino podešavanja.

Za ovaj vodič koristit ćemo jednostavniji format s jednim korakom. Podaci su u [jsonl formatu](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) s 1 zapisom po retku, svaki predstavljen kao JSON objekt. Ispod je prikazano 2 zapisa kao primjer – pogledajte [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) za cijeli skup primjera (10 primjera) koji ćemo koristiti za naš vodič o fino podešavanju. **Napomena:** Svaki zapis _mora_ biti definiran u jednom retku (ne smije biti podijeljen na više redaka kao što je uobičajeno u formatiranom JSON-u)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

U stvarnom slučaju korištenja trebat će vam puno veći skup primjera za dobre rezultate – kompromis će biti između kvalitete odgovora i vremena/troškova za fino podešavanje. Koristimo mali skup kako bismo brzo završili fino podešavanje i prikazali proces. Pogledajte [ovaj primjer iz OpenAI Cookbooka](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) za složeniji vodič o fino podešavanju.


### Korak 1.2 Prenesite svoj skup podataka

Prenesite podatke koristeći Files API [kao što je opisano ovdje](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Imajte na umu da prije pokretanja ovog koda morate prvo napraviti sljedeće korake:
 - Instalirali ste `openai` Python paket (provjerite da koristite verziju >=0.28.0 zbog najnovijih značajki)
 - Postavili ste `OPENAI_API_KEY` varijablu okruženja na svoj OpenAI API ključ
Za više informacija pogledajte [Vodič za postavljanje](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) koji je pripremljen za ovaj tečaj.

Sada pokrenite kod za kreiranje datoteke za prijenos iz vaše lokalne JSONL datoteke.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### Korak 2.1: Kreirajte posao za fino podešavanje pomoću SDK-a


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### Korak 2.2: Provjerite status posla

Evo nekoliko stvari koje možete napraviti s `client.fine_tuning.jobs` API-jem:
- `client.fine_tuning.jobs.list(limit=<n>)` - Prikažite zadnjih n poslova za fino podešavanje
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Dohvatite detalje određenog posla za fino podešavanje
- `client.fine_tuning.jobs.cancel(<job_id>)` - Otkazivanje posla za fino podešavanje
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Prikažite do n događaja s posla
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Prvi korak u procesu je _provjera ispravnosti datoteke za treniranje_ kako biste bili sigurni da su podaci u ispravnom formatu.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### Korak 2.3: Pratite događaje za praćenje napretka


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Korak 2.4: Pogledajte status na OpenAI nadzornoj ploči


Status možete također provjeriti tako da posjetite OpenAI web stranicu i otvorite odjeljak _Fine-tuning_ na platformi. Tamo ćete vidjeti status trenutnog zadatka, kao i povijest prethodnih izvršavanja. Na ovoj snimci zaslona možete vidjeti da je prethodno izvršavanje bilo neuspješno, dok je drugo bilo uspješno. Za kontekst, to se dogodilo jer je prvi pokušaj koristio JSON datoteku s pogrešno formatiranim zapisima – nakon što je to ispravljeno, drugi pokušaj je uspješno dovršen i model je postao dostupan za korištenje.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.hr.png)


Također možete vidjeti statusne poruke i metrike tako da se pomaknete niže u vizualnoj nadzornoj ploči, kao što je prikazano:

| Poruke | Metrike |
|:---|:---|
| ![Poruke](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.hr.png) |  ![Metrike](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.hr.png)|


### Korak 3.1: Dohvatite ID i testirajte fino podešen model u kodu


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### Korak 3.2: Učitajte i testirajte fino podešen model u Playgroundu

Sada možete testirati svoj fino podešeni model na dva načina. Prvo, možete posjetiti Playground i koristiti padajući izbornik Models kako biste odabrali svoj novo fino podešeni model s ponuđenog popisa. Druga opcija je korištenje opcije "Playground" prikazane u panelu za fino podešavanje (pogledajte gornji screenshot), koja pokreće _komparativni_ prikaz u kojem su osnovni i fino podešeni model prikazani jedan pored drugog radi brze procjene.

Jednostavno unesite sistemski kontekst koji ste koristili u svojim podacima za treniranje i postavite svoje testno pitanje. Primijetit ćete da se na obje strane ažurira isti kontekst i pitanje. Pokrenite usporedbu i vidjet ćete razliku u izlazima između njih. _Obratite pažnju kako fino podešeni model daje odgovor u formatu koji ste naveli u svojim primjerima, dok osnovni model samo slijedi sistemski prompt_.

Također ćete primijetiti da usporedba prikazuje broj tokena za svaki model, kao i vrijeme potrebno za izvođenje inferencije. **Ovaj konkretan primjer je jednostavan i služi za prikazivanje procesa, ali ne odražava stvarni skup podataka ili scenarij**. Možda ćete primijetiti da oba uzorka prikazuju isti broj tokena (sistemski kontekst i korisnički upit su identični), dok fino podešeni model troši više vremena na inferenciju (prilagođeni model).

U stvarnim situacijama nećete koristiti ovako jednostavan primjer, već ćete fino podešavati model na stvarnim podacima (npr. katalog proizvoda za korisničku podršku), gdje će kvaliteta odgovora biti puno očitija. U _tom_ kontekstu, postizanje jednake kvalitete odgovora s osnovnim modelom zahtijevat će više prilagođenog prompt inženjeringa, što će povećati potrošnju tokena i potencijalno vrijeme obrade za inferenciju. _Ako želite isprobati ovo, pogledajte primjere fino podešavanja u OpenAI Cookbooku za početak._



---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden pomoću AI usluge prevođenja [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati mjerodavnim izvorom. Za ključne informacije preporučuje se profesionalni ljudski prijevod. Ne snosimo odgovornost za bilo kakva nesporazume ili pogrešna tumačenja koja proizlaze iz korištenja ovog prijevoda.
